<a href="https://colab.research.google.com/github/darisoy/EE517_Sp21/blob/master/Project/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up the environment

In [48]:
# install packages
%%capture
!pip install allennlp
!pip install allennlp-models
!pip install spacy-dbpedia-spotlight

import spacy
from allennlp.predictors.predictor import Predictor
import spacy_dbpedia_spotlight

In [49]:
# install nlp
%%capture
!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')

In [50]:
# initialize models
%%capture
allen_model_url = 'https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz'
allen_predictor = Predictor.from_path(allen_model_url)  # load the model
nel = spacy_dbpedia_spotlight.create('en')

# Functions

In [51]:
def get_coref(text, predictor):
    prediction = predictor.predict(document=text)
    clusters = []
    for cluster in prediction['clusters']:
        mention_ref = []
        for token in cluster:
            token_start = token[0]
            token_end = token[1]
            mention = " ".join(prediction['document'][token_start:token_end + 1])
            mention_ref.append(mention)
        clusters.append(mention_ref)
    return clusters

In [32]:
def get_ner(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == 'PERSON']

In [71]:
def get_nel(text):
    threshold = 0.95
    doc = nel(text)
    return [ent.text for ent in doc.ents if float(ent._.dbpedia_raw_result['@similarityScore']) >= threshold]

# Some sample text

In [87]:
sample = 'Isaac Newton invented the wheel. He didn\'t go to kindergarden but he was familiar with circles. When told this story, Jessica didn\'t believe it. She thought Newton was a lie.'

# Test

In [88]:
clusters = get_coref(sample, allen_predictor)
person_mentions = get_ner(sample)
famous_people = get_nel(sample)
# TODO: get non-name person mentions using simple rules (for he, she, we, you...)
# person_mentions.append(...)

In [89]:
clusters

[['Isaac Newton', 'He', 'he', 'Newton'],
 ['this story', 'it'],
 ['Jessica', 'She']]

In [90]:
person_mentions

['Isaac Newton', 'Jessica', 'Newton']

In [91]:
famous_people

['Isaac Newton', 'Newton']

# Combine results

In [ ]:
# This is PSUEDO code needs work (it looks like this will work, but it won't)
# We should move away from string based system to a word index based model
historical = []
hypothetical = []
for person in person_mentions:
    if person in historical or person in hypothetical: # needs to be fixed, this is not this easy
        continue
    person_set = clusters.index(person) #this is also wrong
    if person in famous_people:
        historical.append(person_set)
    else:
        hypothetical.append(person_set)